# ANALYSE PAR TYPE DES TICKETS

**C'est un notebook où je teste des choses, les codes que j'ai utilisés et que j'ai présentés dans mon ppt et dans mon rapport sont dans le notebook "analyse de la source des tickets"**

### imports des modules et du dataset

In [ ]:
import numpy as np
import os
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from prophet import Prophet
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error, mean_absolute_error
warnings.filterwarnings("ignore")
from pmdarima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
import math
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from pmdarima import auto_arima
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from pandas.plotting import autocorrelation_plot

In [ ]:
os.chdir('C:\\Users\\matthias.picard\\Downloads\\données consort')
statique=pd.read_excel('Static_data.xlsx')

In [ ]:
statique.head()

Nous allons extraire de cette base de données le nombre de tickets par jour/heures/demi-journées. Notre repère temporel sera la colonne Date_deb 

### Préparation des données

#### preparation de la colonne 'Date_deb'

In [ ]:
statique.drop(np.arange(97253,97970,1), inplace=True)

entre les lignes 97253 et 97970 de la colonne Date_deb on ne retrouve que des nan. On est obligé de les enlever pour convertir la colonne en DateTime et s'en servir comme index temporelle

In [ ]:
statique['Date_deb']=pd.to_datetime(statique['Date_deb'])

statique_heures=statique.copy()
statique_heures['Date_deb'] = statique_heures.Date_deb.dt.floor(freq='H') 
statique_heures.set_index('Date_deb',inplace=True)


statique_demijours=statique.copy()
statique_demijours['Date_deb'] = statique_demijours.Date_deb.dt.floor(freq='12h') 
statique_demijours.set_index('Date_deb',inplace=True)


statique_jours=statique.copy()
statique_jours['Date_deb'] = statique_jours.Date_deb.dt.floor(freq='D') 
statique_jours.set_index('Date_deb',inplace=True)

on crée plusieurs dataset selon en regroupant les tickets selon leurs heures/jours/dates de créations

### Dynamique des types de tickets par jours

In [ ]:
statique_jours_demandes=statique_jours[statique_jours['Type_Ticket']=='Demande']
statique_jours_incidents=statique_jours[statique_jours['Type_Ticket']!='Demande']

On va commencer à s'interesser à création de ticket demandes et tickets incidents par jours.

In [ ]:
nb_ticket_jours_demandes=pd.DataFrame(statique_jours_demandes.index.value_counts())
ticket_jours_demandes=nb_ticket_jours_demandes.reset_index().sort_values(by='index').set_index('index').asfreq('d')
ticket_jours_demandes[ticket_jours_demandes.isna()]=0
ticket_jours_demandes.rename(columns={'Date_deb':'nb_tickets'},inplace=True)
ticket_jours_demandes

On obtient ici un dataset du nombre de tickets demandes crée chaques jours entre le 2011/10/17 et le 2020-09-29

In [ ]:
nb_ticket_jours_incidents=pd.DataFrame(statique_jours_incidents.index.value_counts())
ticket_jours_incidents=nb_ticket_jours_incidents.reset_index().sort_values(by='index').set_index('index').asfreq('d')
ticket_jours_incidents[ticket_jours_incidents.isna()]=0
ticket_jours_incidents.rename(columns={'Date_deb':'nb_tickets'},inplace=True)
ticket_jours_incidents

Pareil mais pour les tickets incidents

In [ ]:
plt.figure()

ticket_jours_incidents.plot(c='blue',label='incidents')
ticket_jours_demandes.plot(c='red',label='demandes')
plt.legend()

Le premier graphique montre la répartition des incidents au cours du temps. La dynamique est assez différentes de celle du graphe avec l'ensemble des tickets: la tendance n'est pas linéaire (peu d'incident entre 2016 et 2018), et on peut constater la présence de nombreux pics d'incidents.

Le graphe des demandes ressemble beaucoup plus à celui avec tout les types de tickets ( nottament parce que l'on trouve plus de demande que d'incidents: 96832 pour 29970).

Regardons ce qu'il se passe durant la période ou le nombre d'incident semble anormalement bas:

### Analyse des incidents entre 2016 et 2018

#### value_counts des sources

In [ ]:
sns.barplot(data=pd.DataFrame(statique_jours_incidents.Source.value_counts().reset_index()),
            x='index',
            y='Source',color='red')

In [ ]:
statique_jours_incidents.Source.value_counts()

In [ ]:
sns.barplot(data=pd.DataFrame(statique_jours_incidents.loc['2015/06':'2018/04',:].Source.value_counts().reset_index()),
            x='index',
            y='Source',color='red')

In [ ]:
statique_jours_incidents.loc['2015/06':'2018/04',:].Source.value_counts()

les mails sont quasi-inexistants dans la periode, et on ne trouve personne qui à utiliser MyA

#### value_counts des Entités

In [ ]:
sns.barplot(data=pd.DataFrame(statique_jours_incidents.Entite.value_counts().reset_index()),
            x='index',
            y='Entite',color='blue')

In [ ]:
statique_jours_incidents.Entite.value_counts()

In [ ]:
sns.barplot(data=pd.DataFrame(statique_jours_incidents.loc['2015/06':'2018/04',:].Entite.value_counts().reset_index()),
            x='index',
            y='Entite',color='blue')

In [ ]:
statique_jours_incidents.loc['2015/06':'2018/04',:].Entite.value_counts()

In [ ]:
sns.barplot(statique_jours_incidents.loc['2015/06':'2018/04',:].Entite.hist()

Pas vraiment de changement significatfs lorsque l'on s'interressent aux entités concernées par les incidents signalés dans la période

## tests de modèles

**Pour tout nos modèles on essayera d'appliquer nos prédictions sur la période juillet-septembre (et pas avant, pour éviter la période du confinement, qui n'est pas représentative)**

Commençons par appliquer des modèles Prophet sans tuning aux incidents et aux demandes

### Prophet (simple) __incidents

In [ ]:
ticket_jours_incidents_prophet=ticket_jours_incidents.reset_index().rename(columns={'index':'ds','nb_tickets':'y'})

trainset_simple_incidents_prophet=ticket_jours_incidents_prophet.loc[:'3100',:]
testset_simple_incidents_prophet=ticket_jours_incidents_prophet.loc['3100':,:]


In [ ]:
model_simple_incidents_prophet = Prophet(daily_seasonality=False)
model_simple_incidents_prophet.fit(trainset_simple_incidents_prophet)
future_simple_incidents_prophet = model_simple_incidents_prophet.make_future_dataframe(periods=91)
forecast_simple_incidents_prophet = model_simple_incidents_prophet.predict(future_simple_incidents_prophet)

In [ ]:
model_simple_incidents_prophet.plot(forecast_simple_incidents_prophet)

la période de 2016-2018 trouble la tendance general (il faudra peut-etre la modifier...)

In [ ]:
model_simple_incidents_prophet.plot_components(forecast_simple_incidents_prophet)

In [ ]:
predictions_simple_incidents_prophet=pd.DataFrame(forecast_simple_incidents_prophet.loc[3101:,'yhat'])
predictions_simple_incidents_prophet.set_index(testset_simple_incidents_prophet['ds'],inplace=True)
plt.figure()
predictions_simple_incidents_prophet['yhat'].plot(c='yellow',label='predictions')
plt.plot(testset_simple_incidents_prophet['ds'],testset_simple_incidents_prophet['y'],c='blue',label='vraies valeurs')
plt.legend()

In [ ]:
(mean_squared_error(predictions_simple_incidents_prophet['yhat'],testset_simple_incidents_prophet['y']),
mean_absolute_percentage_error(predictions_simple_incidents_prophet['yhat'],testset_simple_incidents_prophet['y']))

En moyenne 55 % d'erreur entre la prédiction et la vrai valeur....

Sur la detection d'incidents Prophet  semble plutôt bien suivre la tendance géneral au cours de l'année (moins d'incidents en aout, plus d'incident en septembre...). Mais il ne prend pas en compte la saisonnalité par semaine ( il ya très souvent un pic en début de semaine, mais Prophet sous-estime souvent ce pic). Il a aussi souvent tendance à surestimer les incidents le week-end (augmenter l'impact de la saisonnalité par semaine?)

On peut aussi remarquer que ces pics sont beaucoup moins présent le mois d'août. Doit-on rajouter une saisonnalité liée aux vacances? 

### Prophet (simple)__demandes

In [ ]:
ticket_jours_demandes_prophet=ticket_jours_demandes.reset_index().rename(columns={'index':'ds','nb_tickets':'y'})

trainset_simple_demandes_prophet=ticket_jours_demandes_prophet.loc[:'3150',:]
testset_simple_demandes_prophet=ticket_jours_demandes_prophet.loc['3150':,:]

In [ ]:
model_simple_demandes_prophet = Prophet(daily_seasonality=False)
model_simple_demandes_prophet.fit(trainset_simple_demandes_prophet)
future_simple_demandes_prophet = model_simple_demandes_prophet.make_future_dataframe(periods=121)
forecast_simple_demandes_prophet = model_simple_demandes_prophet.predict(future_simple_demandes_prophet)

In [ ]:
model_simple_demandes_prophet.plot(forecast_simple_demandes_prophet)

In [ ]:
model_simple_demandes_prophet.plot_components(forecast_simple_demandes_prophet)

In [ ]:
predictions_simple_demandes_prophet=pd.DataFrame(forecast_simple_demandes_prophet.loc[3151:,'yhat'])
predictions_simple_demandes_prophet.set_index(testset_simple_demandes_prophet['ds'],inplace=True)
plt.figure()
predictions_simple_demandes_prophet['yhat'].plot(c='yellow',label='predictions')
plt.plot(testset_simple_demandes_prophet['ds'],testset_simple_demandes_prophet['y'],c='blue',label='vraies valeurs')
plt.legend()


In [ ]:
(mean_squared_error(predictions_simple_demandes_prophet['yhat'],testset_simple_demandes_prophet['y']),
mean_absolute_percentage_error(predictions_simple_demandes_prophet['yhat'],testset_simple_demandes_prophet['y']))

60% d'erreurs en moyenne.

Ici, Prophet surestime considérablment la quantités de demandes crée pendant les week-end

### Prophet tunné__demandes

On va tenter d'améliorer notre prédiction en jouant sur les paramètres suivants: 
- saisonnalité mensuelle
- indiquer les jours de vacances
- traiter les périodes exceptionnelles ( confinement)

In [ ]:
ticket_jours_demandes_prophet=ticket_jours_demandes.reset_index().rename(columns={'index':'ds','nb_tickets':'y'})

trainset_tunné_demandes_prophet=ticket_jours_demandes_prophet.loc[:'3150',:]
testset_tunné_demandes_prophet=ticket_jours_demandes_prophet.loc['3150':,:]

In [ ]:
model_tunné_demandes_prophet= Prophet(growth='linear',yearly_seasonality=False
                 ,seasonality_prior_scale=25, weekly_seasonality=False
                 ).add_seasonality(name='yearly',period=365.25,fourier_order=10
                                  ).add_seasonality(name='weekly',period=7,fourier_order=10,prior_scale=50)

model_tunné_demandes_prophet.fit(trainset_tunné_demandes_prophet)
future_tunné_demandes_prophet= model_tunné_demandes_prophet.make_future_dataframe(periods=121)
forecast_tunné_demandes_prophet= model_tunné_demandes_prophet.predict(future_tunné_demandes_prophet)


model_tunné_demandes_prophet.plot(forecast_tunné_demandes_prophet)

model_tunné_demandes_prophet.plot_components(forecast_tunné_demandes_prophet)

In [ ]:
predictions_tunné_demandes_prophet=pd.DataFrame(forecast_tunné_demandes_prophet.loc[3151:,'yhat'])
predictions_tunné_demandes_prophet.set_index(testset_tunné_demandes_prophet['ds'],inplace=True)
plt.figure()
predictions_tunné_demandes_prophet['yhat'].plot(c='yellow',label='predictions')
plt.plot(testset_tunné_demandes_prophet['ds'],testset_tunné_demandes_prophet['y'],c='blue',label='vraies valeurs')
plt.legend()

In [ ]:
(mean_squared_error(predictions_tunné_demandes_prophet['yhat'],testset_tunné_demandes_prophet['y']),
mean_absolute_percentage_error(predictions_tunné_demandes_prophet['yhat'],testset_tunné_demandes_prophet['y']))

#### (Voir comment améliorer ces modèles avec Vladimir)

### SARIMA et auto-arima

#### test de stationnarité

In [ ]:
result_incidents = adfuller(ticket_jours_incidents['nb_tickets'].values)
result_demandes = adfuller(ticket_jours_demandes['nb_tickets'].values)
print(result_incidents[1],result_demandes[1])

Dans les deux cas la p-value est assez négligeable, on peut donc considerer les données comme étant stationnaires